In [1]:
import pandas as pd
import numpy as np
import spotipy
import enchant
import time
from spotipy.oauth2 import SpotifyClientCredentials

# Stage 1: Data Retrieval 
In this stage, I am retrieving Spotify music data from the Spotify Web API, with the help of a Python library named "Spotipy", which is a library that makes data scrapping easier. I mainly followed this tutorial that I found to retrieve the data: https://rareloot.medium.com/extracting-spotify-data-on-your-favourite-artist-via-python-d58bc92a4330

I have already created an account and created an app on the Spotify Web API, which is a requirement to access the API. Here, I am creating client_id, and client_secret_key provided after I have registered with Spotify for Developers. The following steps authenticate requests. Besides, I am also only retrieving the data that I need, and my purpose for retrieving this data is to create new value from it, not to duplicate it. 

This file also only demonstrates data retrieval for exactly 1 artist. For the sake of data cleaning, I am retrieving data for one artist at a time in order to save each artist's songs as an individual file. I can retrieve music data of another artist by making just 2 simple changes. 

In [2]:
client_id = "24122081564347bdbac5e0064c120fc3"
client_secret_key = "e0c98c773ab8430fb144abaa9231be12"

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret_key)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

I am interested in the following 6 artists and all songs released since debut:

In [5]:
bts = 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'
shinee = 'https://open.spotify.com/artist/2hRQKC0gqlZGPrmUKbcchR'
redvelvet = 'https://open.spotify.com/artist/1z4g3DjTBBZKhvAroFlhOM'
exo = 'https://open.spotify.com/artist/3cjEqqelV9zb4BYE3qDQ4O'
twice = 'https://open.spotify.com/artist/7n2Ycct7Beij7Dj7meI4X0'
blackpink = 'https://open.spotify.com/artist/41MozSoPIsD1dJM0CLPjZF'

artist_names = [redvelvet]

artist_names_str = ['red velvet'] # change the item in the list to scrape one artist at a time

album_names = []
album_uris = []
each_artist_album = []
album_release_date = []

for i in range(len(artist_names)): 
    artist_name = artist_names[i] 
    artist_name_str = artist_names_str[i] 
    spotify_albums = sp.artist_albums(artist_id = artist_name, limit = 50)

    for i in range(len(spotify_albums['items'])):
        album_names.append(spotify_albums['items'][i]['name'])
        album_uris.append(spotify_albums['items'][i]['uri'])
        each_artist_album.append(artist_name_str) 
        album_release_date.append(spotify_albums['items'][i]['release_date']) 

In [6]:
def albumSongs(uri, name_of_artist, release_date):
    album = uri 
    spotify_albums[album] = {}
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    spotify_albums[album]['artist'] = []
    spotify_albums[album]['release_date'] = []
    tracks = sp.album_tracks(album)
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])
        spotify_albums[album]['release_date'].append(release_date)
        spotify_albums[album]['artist'].append(name_of_artist)

In [7]:
spotify_albums = {}
album_count = 0

for i in range(len(album_uris)):
    album_uri = album_uris[i]
    albumSongs(album_uri, each_artist_album[i], album_release_date[i])
    album_count += 1

In [8]:
def get_audio_features(album):
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    track_count = 0
    for track in spotify_albums[album]['uri']:
        features = sp.audio_features(track)
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [10]:
slp_min = 2
slp_max = 5
start_time = time.time()
rq_count = 0
for i in spotify_albums:
    if i is None:
        continue
    get_audio_features(i)
    rq_count+=1
    if rq_count % 10 == 0:
        time.sleep(np.random.uniform(slp_min, slp_max))

In [12]:
dic_df = {}
dic_df['album'] = []
dic_df['release_date'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
dic_df['artist'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])

Here is a glimpse of the data frame that is retrieved:

In [13]:
df = pd.DataFrame.from_dict(dic_df)
df

,album,release_date,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,artist
0,‘The ReVe Festival’ Finale,2019-12-23,1,3CYH422oy1cZNoo0GTG1TK,Psycho,spotify:track:3CYH422oy1cZNoo0GTG1TK,0.1930,0.664,0.666,0.000000,0.0854,-2.203,0.0446,139.987,0.405,80,red velvet
1,‘The ReVe Festival’ Finale,2019-12-23,2,2iYnEfKsAsE86Yb9mS4b3E,In & Out,spotify:track:2iYnEfKsAsE86Yb9mS4b3E,0.2110,0.568,0.804,0.000000,0.1400,-2.995,0.1360,159.680,0.612,62,red velvet
2,‘The ReVe Festival’ Finale,2019-12-23,3,2m9K7KlDOQ0NXk9xBpkuoZ,Remember Forever,spotify:track:2m9K7KlDOQ0NXk9xBpkuoZ,0.5700,0.576,0.619,0.000000,0.0916,-3.324,0.0419,149.896,0.524,54,red velvet
3,‘The ReVe Festival’ Finale,2019-12-23,4,5vdqSjUj9JZsAHdr02Vygk,"Eyes Locked, Hands Locked",spotify:track:5vdqSjUj9JZsAHdr02Vygk,0.6240,0.647,0.674,0.000001,0.2020,-3.236,0.0441,79.986,0.734,52,red velvet
4,‘The ReVe Festival’ Finale,2019-12-23,5,1wpAS4Jm9ElkOJjnyDX4Tm,Ladies Night,spotify:track:1wpAS4Jm9ElkOJjnyDX4Tm,0.0895,0.598,0.883,0.000000,0.0731,-3.456,0.0792,115.023,0.705,50,red velvet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Hotel Del Luna OST,2019-08-31,9,6YStjHestZIG4UB4SKCVsQ,Can you hear me?,spotify:track:6YStjHestZIG4UB4SKCVsQ,0.8200,0.518,0.587,0.000000,0.3390,-4.523,0.0326,137.575,0.152,29,red velvet
205,Hotel Del Luna OST,2019-08-31,10,4HCKhlRABrYBQGQmpABgCn,So long,spotify:track:4HCKhlRABrYBQGQmpABgCn,0.8090,0.450,0.292,0.000000,0.1600,-7.090,0.0312,131.151,0.292,31,red velvet
206,Hotel Del Luna OST,2019-08-31,11,1j4OVltb8tHL90IrU6BL3L,Say Goodbye,spotify:track:1j4OVltb8tHL90IrU6BL3L,0.8450,0.535,0.449,0.000000,0.1260,-6.014,0.0272,139.723,0.300,28,red velvet
207,Hotel Del Luna OST,2019-08-31,12,0UEufwQxHpf7Y9XE33uSNr,Done For Me,spotify:track:0UEufwQxHpf7Y9XE33uSNr,0.7100,0.498,0.454,0.000000,0.1110,-8.072,0.0396,135.211,0.228,33,red velvet


In [14]:
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()

In [15]:
final_df.to_csv("rv_songs.csv")

### Summary of this stage:

- I used the Spotify Web API which provided the data I was looking for, generated a unique client ID and a client secret key, and used the Python library "Spotipy" which allowed me to get access to Spotify data for songs, artists, and various attributes of each song, including its title, album, date released, popularity score, danceability score, tempo score, speechiness score, liveness score, and a variety of other numeric scores related to the musical features of a song. 
- I retrieved data of a total of 6 popular k-pop groups, each group’s songs data as a separate csv file, including BTS, EXO, Twice, Red Velvet, Shinee, and Blackpink. Each csv file has 18 columns, and each row represents a song. 
- A brief glance of the format of the resulting dataset is shown in this file.

### Note:

- This stage, data retrieval, does not involve any manipulation of the data yet. It is only for retrieving the data off the web API for each artist I am retrieving data for.